# 数组读写
## 从文本中读写数组

In [1]:
import numpy as np

### 空格（制表符）分割的文本
假设我们有这样的一个空白分割的文件：

In [2]:
%%writefile myfile.txt
2.1 2.3 3.2 1.3 3.1
6.1 3.1 47.2 2.3 1.8

Writing myfile.txt


为了生成数组，我们首先将数组转换成一个列表组成的列表，再将这个列表转换为数组：

In [4]:
data = []
with open('myfile.txt') as f:
    for line in f:
        fileds = line.split()
        row_data = [float(x) for x in fileds]
        data.append(row_data)
        
data = np.array(data)

In [5]:
data

array([[ 2.1,  2.3,  3.2,  1.3,  3.1],
       [ 6.1,  3.1, 47.2,  2.3,  1.8]])

不过，更简便的是使用loadtxt方法：

In [6]:
data  = np.loadtxt('myfile.txt')
data

array([[ 2.1,  2.3,  3.2,  1.3,  3.1],
       [ 6.1,  3.1, 47.2,  2.3,  1.8]])

### 逗号分隔文件  

In [7]:
%%writefile myfile.txt
2.1,2.3,3.2,1.3,3.1
6.1,3.1,4.2,2.3,1.8

Overwriting myfile.txt


对于逗号分隔的文件（通常是.csv格式），我们可以稍微修改之前繁琐的过程，将split的参数变成','即可。

不过，loadtxt函数也可以读这样的文件，只需要制定分隔符的参数即可。

In [9]:
data = np.loadtxt('myfile.txt',delimiter=',')
data

array([[2.1, 2.3, 3.2, 1.3, 3.1],
       [6.1, 3.1, 4.2, 2.3, 1.8]])

### loadtxt函数
```py
loadtxt(fname, dtype=<type 'float'>, 
        comments='#', delimiter=None, 
        converters=None, skiprows=0, 
        usecols=None, unpack=False, ndmin=0)
```
loadtxt 有很多可选参数，其中 delimiter 就是刚才用到的分隔符参数。

skiprows 参数表示忽略开头的行数，可以用来读写含有标题的文本

In [11]:
%%writefile myfile.txt
X Y Z MAG ANG
2.1 2.3 3.2 1.3 3.1
6.1 3.1 4.2 2.3 1.8

Overwriting myfile.txt


In [12]:
np.loadtxt('myfile.txt',skiprows=1)

array([[2.1, 2.3, 3.2, 1.3, 3.1],
       [6.1, 3.1, 4.2, 2.3, 1.8]])

此外，有一个功能更为全面的 genfromtxt 函数，能处理更多的情况，但相应的速度和效率会慢一些。
```
genfromtxt(fname, dtype=<type 'float'>, comments='#', delimiter=None, 
           skiprows=0, skip_header=0, skip_footer=0, converters=None, 
           missing='', missing_values=None, filling_values=None, usecols=None, 
           names=None, excludelist=None, deletechars=None, replace_space='_', 
           autostrip=False, case_sensitive=True, defaultfmt='f%i', unpack=None, 
           usemask=False, loose=True, invalid_raise=True)
```
### loadtxt 的更多特性
对于这样一个文件

In [13]:
%%writefile myfile.txt
 -- BEGINNING OF THE FILE
% Day, Month, Year, Skip, Power
01, 01, 2000, x876, 13 % wow!
% we don't want have Jan 03rd
04, 01, 2000, xfed, 55

Overwriting myfile.txt


In [15]:
data = np.loadtxt('myfile.txt',
                 skiprows=1,
                 dtype=np.int,
                 delimiter=',',
                 usecols=(0,1,2,4),
                 comments='%')
data

array([[   1,    1, 2000,   13],
       [   4,    1, 2000,   55]])

### 自定义转换方法

In [16]:
%%writefile myfile.txt
2010-01-02 2.3 3.1
2011-01-01 6.1 3.1

Overwriting myfile.txt


假设我们的文本包含日期，我们可以使用datetime在loadtxt中处理：

In [23]:
import datetime

def date_converter(s):
    return datetime.datetime.strptime(s, "%Y-%m-%d")

data = np.loadtxt('myfile.txt',
                  dtype=np.object, #数据类型为对象
                  converters={0:date_converter,  #第一列使用自定义转换方法
                              1:str,           #第二第三使用浮点数转换
                              2:float})
data

TypeError: strptime() argument 1 must be str, not bytes

In [24]:
import os
os.remove('myfile.txt')

## 将数组写入文件
savetxt 可以将数组写入文件，默认使用科学计数法的形式保存：


In [25]:
data = np.array([[1,2],
                [3,4]])
np.savetxt('out.txt',data)

In [29]:
with open('out.txt') as f:
    for line in f:
        print (line+',')

1.000000000000000000e+00 2.000000000000000000e+00
,
3.000000000000000000e+00 4.000000000000000000e+00
,


也可以使用类似C语言printf的方式制定输出格式：

In [33]:
data = np.array([[1,2], 
                 [3,4]])

np.savetxt('out.txt', data, fmt="%d") #保存为整数

In [34]:
with open('out.txt') as f:
    for line in f:
        print (line)

1 2

3 4



逗号分隔的输出：

In [35]:
data = np.array([[1,2], 
                 [3,4]])

np.savetxt('out.txt', data, fmt="%.2f", delimiter=',') #保存为2位小数的浮点数，用逗号分隔

In [37]:
with open('out.txt') as f:
    for line in f:
        print (line)

1.00,2.00

3.00,4.00



复数值默认会加上括号：

In [38]:
data = np.array([[1+1j,2], 
                 [3,4]])

np.savetxt('out.txt', data, fmt="%.2f", delimiter=',') #保存为2位小数的浮点数，用逗号分隔

In [40]:
with open('out.txt') as f:
    for line in f:
        print (line)

 (1.00+1.00j), (2.00+0.00j)

 (3.00+0.00j), (4.00+0.00j)



更多参数：
```
savetxt(fname, 
        X, 
        fmt='%.18e', 
        delimiter=' ', 
        newline='\n', 
        header='', 
        footer='', 
        comments='# ')
```
移除 out.txt：

In [42]:
import os
os.remove('out.txt')

## Numpy 二进制格式
数组可以储存成二进制格式，单个的数组保存为 .npy 格式，多个数组保存为多个.npy文件组成的 .npz 格式，每个 .npy 文件包含一个数组。

与文本格式不同，二进制格式保存了数组的 shape, dtype 信息，以便完全重构出保存的数组。

保存的方法：

- save(file, arr) 保存单个数组，.npy 格式
- savez(file, *args, **kwds) 保存多个数组，无压缩的 .npz 格式
- savez_compressed(file, *args, **kwds) 保存多个数组，有压缩的 .npz 格式
读取的方法：

load(file, mmap_mode=None) 对于 .npy，返回保存的数组，对于 .npz，返回一个名称-数组对组成的字典。
### 单个数组的读写

In [44]:
a = np.array([[1.0,2.0],
             [3.0,4.0]])
fname = 'afile.npy'
np.save(fname,a)

In [45]:
aa = np.load(fname)
aa

array([[1., 2.],
       [3., 4.]])

删除生成的文件：

In [46]:
import os
os.remove('afile.npy')

### 二进制与文本大小比较

In [47]:
a = np.arange(10000.)

保存为文本

In [48]:
np.savetxt('a.txt',a)

查看大小

In [50]:
import os
os.stat('a.txt').st_size

250000

保存为二进制文件

In [51]:
np.save('a.npy',a)

查看大小

In [53]:
os.stat('a.npy').st_size

80128

删除生成的文件：

In [56]:
os.remove('a.npy')
os.remove('a.txt')

可以看到二进制文件大约是文本文件的三分之一

### 保存多个数组

In [58]:
a = np.array([[1.0,2.0], 
              [3.0,4.0]])
b = np.arange(1000)

In [59]:
np.savez('data.npz', a=a, b=b)

查看里面包含的文件：

In [60]:
!unzip -l data.npz

Archive:  data.npz
  Length      Date    Time    Name
---------  ---------- -----   ----
      160  1980-01-01 00:00   a.npy
     8128  1980-01-01 00:00   b.npy
---------                     -------
     8288                     2 files


载入数据：

In [61]:
data = np.load('data.npz')

载入后可以像字典一样进行操作：

In [62]:
data.keys()

KeysView(<numpy.lib.npyio.NpzFile object at 0x7f5b28154bd0>)

In [63]:
data['a']

array([[1., 2.],
       [3., 4.]])

In [64]:
data['b'].shape

(1000,)

删除文件：

In [65]:
# 要先删除 data，否则删除时会报错
del data

os.remove('data.npz')

## 压缩文件
当数据比较整齐时：

In [67]:
a = np.arange(20000.)

无压缩大小：

In [68]:
np.savez('a.npz', a=a)
os.stat('a.npz').st_size

160256

有压缩大小：

In [69]:
np.savez_compressed('a2.npz', a=a)
os.stat('a2.npz').st_size

26908

大约有 6x 的压缩效果。

当数据比较混乱时：

In [70]:
a = np.random.rand(20000.)

TypeError: 'float' object cannot be interpreted as an integer

无压缩大小：

In [72]:
np.savez('a.npz', a=a)
os.stat('a.npz').st_size

160256

有压缩大小：

In [73]:
np.savez_compressed('a2.npz', a=a)
os.stat('a2.npz').st_size

26908

只有大约 1.06x 的压缩效果。

In [74]:
os.remove('a.npz')
os.remove('a2.npz')